In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import matplotlib.pyplot as plt
import os

"""
Directories :
/kaggle/input
 - happy-whale-and-dolphin
   sample_submission.csv
   train.csv
   - train_images
   - test_images
"""

train_file = '/kaggle/input/whale2-cropped-dataset/train2.csv'
test_file = '/kaggle/input/whale2-cropped-dataset/test2.csv'
submission_file = '/kaggle/working/submission.csv'

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Mise en place des scripts importées depuis notre GitHub

In [ ]:
os.chdir('/kaggle/working/')
if os.path.exists('/kaggle/working/HappyWhale'):
    !rm -rf /kaggle/working/HappyWhale/
!git clone https://github.com/julien-levarlet/HappyWhale.git
os.chdir('/kaggle/working/HappyWhale')
!pip install -r requirements.txt

In [ ]:
import torch.nn as nn
import torch
from src.ModelTrainTestManager import ModelTrainTestManager, optimizer_setup
from src.DataManagerKaggle import DataManager
from src.Models.HappyWhaleModel import HappyWhaleModel
#from src.yolo import Yolo_Transformation

dataFolderPath = "/kaggle/input/whale2-cropped-dataset"

## Crop images with Detic
We will use directily cropped images from whale2-cropped-dataset, because our Yolo implementation is too demanding in ram memory

## Train the model on cropped images

In [ ]:
batch_size = 10
learning_rate = 0.01
optimizer_factory = optimizer_setup(torch.optim.Adam, lr=learning_rate)

test_percentage = 0.0
val_percentage = 0.1

exp_name = "HappyWhale"

print(test_file, train_file, dataFolderPath)
data_manager = DataManager(train_file, test_file, dataFolderPath, batch_size,
                           val_percentage, verbose=True)


In [ ]:
in_channels=3
depth=3
option="small"
size=256
epoch=8
num_class = data_manager.get_number_of_classes()

# ArcFace Hyperparameters
arcFace_config = {
    "s": 30.0,  # scale (The scale parameter changes the shape of the logits. The higher the scale, the more peaky the logits vector becomes.)
    "m": 0.50,  # margin (margin results in a bigger separation of classes in your training set)
    "ls_eps": 0.0,
    "easy_margin": False
}

model = HappyWhaleModel("tf_efficientnet_b0_ns", embedding_size=512, num_class=num_class, arcface_config=arcFace_config)

In [ ]:
model_trainer = ModelTrainTestManager(model=model,
                                        data_manager = data_manager,
                                        loss_fn=nn.CrossEntropyLoss(),
                                        optimizer_factory=optimizer_factory,
                                        exp_name = exp_name ,
                                        learning_rate=learning_rate,
                                        use_cuda=True)

model_trainer.train(epoch)
model_trainer.get_embeddings_on_test_set()
model_trainer.inference_for_test_set()


In [ ]:
model_trainer.saveTest(submission_file)